Importing all the library

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import seaborn as sns

from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.metrics import confusion_matrix, f1_score

In [4]:
# Disable oneDNN optimizations (optional)
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

 Define the 7 classes and mapping from class name to integer label

In [5]:

class_names = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
num_classes = len(class_names)
label_mapping = {name: idx for idx, name in enumerate(class_names)}

Paths for images & metadata

In [ ]:

train_image_dir = "C:\\Users\\arjun\\OneDrive\\Desktop\\2Train"
train_metadata_path = "C:\\Users\\arjun\\OneDrive\\Desktop\\Train_metadata.csv"
val_image_dir = "C:\\Users\\arjun\\OneDrive\\Desktop\\Validation"
val_metadata_path = "C:\\Users\\arjun\\OneDrive\\Desktop\\2Validation_metadata.csv"
test_image_dir = "C:\\Users\\arjun\\OneDrive\\Desktop\\Test"
test_metadata_path = "C:\\Users\\arjun\\OneDrive\\Desktop\\Test_metadata.csv"


load & preprocess an image given its path and dx label

In [ ]:

def load_image_label(image_path, dx):
    try:
        image_data = tf.io.read_file(image_path)
        image_data = tf.image.decode_jpeg(image_data, channels=3)  # Load image
        image_data = tf.image.resize(image_data, [256, 256])       # Ensure it is 256x256
        image_data = tf.image.convert_image_dtype(image_data, tf.float32)  # Normalize to [0, 1]
        dx = tf.cast(dx, tf.int32)
        return image_data, dx
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return tf.zeros([256, 256, 3], dtype=tf.float32), tf.constant(-1, dtype=tf.int32)


Create a tf.data.Dataset from metadata CSV and image directory using slicing

In [ ]:


def create_dataset(image_dir, metadata_path, batch_size=32, shuffle=True, sample_count=None):
    df = pd.read_csv(metadata_path)
    # If sample_count is provided, slice the dataframe
    if sample_count is not None:
        df = df.iloc[:sample_count]
    # Map string dx values to integer labels if necessary
    if df['dx'].dtype == object:
        df['dx'] = df['dx'].map(label_mapping)
    image_paths = df['image_id'].apply(lambda x: os.path.join(image_dir, x + ".jpg")).tolist()
    labels = df['dx'].tolist()
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.map(lambda path, dx: load_image_label(path, dx),
                num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=12000, seed=42)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

Create datasets for training, validation, and testing

In [1]:

batch_size = 32
train_ds = create_dataset(train_image_dir, train_metadata_path, batch_size=batch_size, shuffle=True, sample_count=12000)
val_ds   = create_dataset(val_image_dir, val_metadata_path, batch_size=batch_size, shuffle=False, sample_count=6000)
test_ds  = create_dataset(test_image_dir, test_metadata_path, batch_size=batch_size, shuffle=False, sample_count=5000)


NameError: name 'create_dataset' is not defined

Data augmentation for the training dataset (modified for lower intensity)

In [ ]:

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # Reduced brightness and contrast augmentation
    image = tf.image.random_brightness(image, max_delta=0.05)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    # Limit rotation to 0° or 90° (instead of 0°, 90°, 180°, 270°)
    k = tf.random.uniform(shape=[], minval=0, maxval=2, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    return image, label

train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
# Debug: Check a batch's shapes
for img_batch, dx_batch in train_ds.take(1):
    print(f"Image batch shape: {img_batch.shape}")   # Expected: (32, 256, 256, 3)
    print(f"DX batch shape: {dx_batch.shape}")         # Expected: (32,)


Build an enhanced CNN model for multi-class classification using softmax.
Adjusted architecture to reduce over-regularization and underfitting.

In [ ]:

def build_cnn(input_shape, num_classes):
    # 0.0005 L2 regularization factor
    l2_reg = tf.keras.regularizers.l2(0.0005)
    model = models.Sequential([
        Input(shape=input_shape),
        # Block 1
        layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.2),

        # Block 2
        layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.2),

        # Block 3
        layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.2),

        # Block 4 (deeper layer)
        layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=l2_reg),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax', kernel_regularizer=l2_reg)
    ])
    return model

input_shape = (256, 256, 3)
cnn_model = build_cnn(input_shape, num_classes)
cnn_model.summary()


Compile the model without precision and recall metrics

In [ ]:

cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", SparseCategoricalAccuracy()]
)

Early stopping callback based on validation loss (increased patience)

In [ ]:

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# ---------------------------------------
# Learning Rate Scheduler callback to reduce LR when validation loss plateaus
# ---------------------------------------
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-06)


Train the model

In [2]:

EPOCHS = 150
history = cnn_model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=EPOCHS, 
    callbacks=[early_stopping, reduce_lr], 
    verbose=1
)

NameError: name 'cnn_model' is not defined

Save the trained model as a Keras file

In [ ]:

cnn_model.save("C:\\Users\\arjun\\OneDrive\\Desktop\\CNN_Model.keras")


Evaluate the model on the test dataset

In [ ]:

results = cnn_model.evaluate(test_ds)
for name, value in zip(cnn_model.metrics_names, results):
    print(f"Test {name}: {value:.4f}")


Plot training vs. validation curves for loss and metrics (excluding precision and recall)

In [1]:

metrics_to_plot = ["loss", "accuracy", "sparse_categorical_accuracy"]
plt.figure(figsize=(15, 5))
for i, metric in enumerate(metrics_to_plot):
    plt.subplot(1, 3, i+1)
    plt.plot(history.history[metric], label=f"Train {metric}")
    plt.plot(history.history[f"val_{metric}"], label=f"Val {metric}")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.title(f"Train vs Validation {metric}")
    plt.legend()
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

Generate predictions on the test set and compute a confusion matrix

In [ ]:

y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)
y_pred_probs = cnn_model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

Calculate F1 Score manually using sklearn

In [ ]:

f1 = f1_score(y_true, y_pred, average='macro')
print(f"Test F1 Score (Macro Average): {f1:.4f}")
